# **응급상황 자동 인식 및 응급실 연계 서비스**
# **단계4 : 통합 - pipeline**

## **0.미션**

단계 4에서는, 단계1,2,3 에서 생성한 함수들을 모듈화하고, 단위 테스트 및 파이프라인 코드를 작성합니다.

* **미션6**
    * 단위 테스트
        * 각 기능(함수)에 대해 단계별로 테스트를 수행하며 오류를 해결합니다.
    * 파이프라인 구축
        * 단계1의 결과가 단계2 모델에 input이 되고, 모델의 예측 결과를 기반으로
        * 응급실 추천되도록
        * 조원들이 녹음한 음성 파일에 임의의 좌표(위도, 경도)값을 부여
            * 음성파일 이름과 좌표를 저장하는 별도 데이터셋 생성
        * 각 모듈을 연결하여 파이프라인 구성하는 ipynb 파일 생성



## **1.환경설정**

### (1) 경로 설정

구글 드라이브 연결

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(project6_2)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
path = '/content/drive/MyDrive/project6-2/'

### (2) 라이브러리

#### 2) 라이브러리 로딩

In [17]:
#필요한 라이브러리 설치 및 불러우기
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # 경고메시지 안 뜨게
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
sys.path.append('/content/drive/MyDrive/project6-2')
from module import emergency



## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [18]:
module_path = path + 'module/'

In [19]:
module_path

'/content/drive/MyDrive/project6-2/module/'

In [20]:
client = emergency.init_openai()
print(client)

### (2) audio to text

In [21]:
audio_path = module_path + "audio/"
audio_file = open(audio_path + "5단계_약_처방_요청_gh.wav", "rb")
print(audio_file)

<_io.BufferedReader name='/content/drive/MyDrive/project6-2/module/audio/5단계_약_처방_요청_gh.wav'>


In [22]:
text = emergency.audio_to_text(client,audio_file)
text

'저희 할머니가 드시는 약이 떨어졌는데 다시 처방받아야 할 것 같아요. 어떻게 해야 할까요?'

### (3) text summary

In [23]:
summarized_text = emergency.summarize_text(client,text)
summarized_text

{'summary': '할머니가 드시는 약이 떨어져서 다시 처방받아야 하는 상황입니다.',
 'keywords': '할머니, 약, 떨어짐, 다시 처방, 상황'}

### (4) 응급실 등급분류

In [24]:
#모델 로드
tokenizer, model, device=emergency.load_emergency_model()

In [25]:
#가져온 모델로 해당 유저의 경도 예측
emergency_grade,percent=emergency.predict_emergency(summarized_text,tokenizer, model, device)
print(emergency_grade,percent)

4 tensor([0.2400, 0.1475, 0.1897, 0.0133, 0.4095])


### (5) 응급실추천

In [26]:

# 테스트를 위한 좌표 설정 (예: 서울시청 좌표)
# user_lat, user_lon = 37.5665, 126.9780
user_lat, user_lon = 35.1146979145937, 129.036828325565
#응급상황 판단 (기준은 등급 <=3)
if(emergency_grade+1<=3):
  result=emergency.recommend_nearest_hospitals(user_lat, user_lon, emergency.get_hospital_data())
  print(result)
else:
  result = ''
  print("응급 상황이 아닙니다")


응급 상황이 아닙니다


## **3. 파이프라인**

* 세부사항
    * [2. 단계별 테스트] 의 내용을 순차적으로 정리합니다.
        * 데이터 처리 전 준비작업 : 한번 실행하면 되는 영역
            * 키, 데이터로딩
            * 모델/토크나이저 로딩
        * 입력값이 들어 왔을 때 출력값까지 처리되는 영역

In [27]:
module_path = path + 'module/'
audio_path = module_path + "audio/"
audio_file = open(audio_path + "5단계_약_처방_요청_gh.wav", "rb")
print(audio_file)

<_io.BufferedReader name='/content/drive/MyDrive/project6-2/module/audio/5단계_약_처방_요청_gh.wav'>


In [28]:
result=emergency.recommend_hospital(audio_file, user_lat, user_lon)
result

병원 추천이 필요없습니다.
